In [66]:
import os 
from pathlib import Path

In [67]:
%pwd

'C:\\Users\\Shyam\\OneDrive\\Desktop\\Kidney-disease-Classification'

In [68]:
os.chdir(Path("C:/Users/Shyam/OneDrive/Desktop/Kidney-disease-Classification/"))     

In [69]:
%pwd

'C:\\Users\\Shyam\\OneDrive\\Desktop\\Kidney-disease-Classification'

In [70]:
from dataclasses import dataclass
from pathlib import Path      


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [71]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
     def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

     def get_data_ingestion_config(self) -> DataIngestionConfig:
         config = self.config.data_ingestion
         create_directories([config.root_dir])
         data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
         )
         return data_ingestion_config


In [72]:
import os
import zipfile
import gdown
from cnnClassifier import logger


In [75]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir.as_posix())

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [76]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-08-13 21:19:28,356: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-13 21:19:28,358: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-13 21:19:28,360: INFO: common: created directory at: artifacts]
[2025-08-13 21:19:28,360: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-13 21:19:28,360: INFO: 3277490812: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts\data_ingestion\data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=5b0eea54-e5e9-48c4-baad-42606500e75f
To: C:\Users\Shyam\OneDrive\Desktop\Kidney-disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:13<00:00, 4.38MB/s]

[2025-08-13 21:19:45,798: INFO: 3277490812: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts\data_ingestion\data.zip]
